# Intro

In [5]:
from tinygrad import Tensor, Context
a = Tensor.empty(4,4)
b = Tensor.empty(4,4)

print((a+b).tolist())

[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]


In [6]:
print(a+b)

<Tensor <UOp METAL (4, 4) float (<Ops.ADD: 62>, None)> on METAL with grad None>


Lazy computation so only computed answer if tolist numpy or realize on tensor are used

In [11]:
with Context(DEBUG=4, NOOPT=True): 
    a = Tensor.empty(4,4)
    b = Tensor.empty(4,4)
    print((a+b).tolist())
    print((a.sum(0).tolist()))

#include <metal_stdlib>
using namespace metal;
kernel void E_16(device float* data0_16, device float* data1_16, device float* data2_16, uint3 gid [[threadgroup_position_in_grid]], uint3 lid [[thread_position_in_threadgroup]]) {
  int gidx0 = gid.x; /* 16 */
  float val0 = (*(data1_16+gidx0));
  float val1 = (*(data2_16+gidx0));
  *(data0_16+gidx0) = (val0+val1);
}
*** METAL      4 E_16                                         arg  3 mem  0.00 GB tm      7.25us/     0.02ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['tolist', '__add__', 'empty']
[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]
#include <metal_stdlib>
using namespace metal;
kernel void r_4_4n1(device float* data0_4, device float* data1_16, uint3 gid [[threadgroup_position_in_grid]], uint3 lid [[thread_position_in_threadgroup]]) {
  float acc0[1];
  int gidx0 = gid.x; /* 4 */
  *(acc0+0) = 0.0f;
  for (int ridx1001 = 0; ridx1001 < 4; ridx1001++) {
    float val0 = (*(data1_16+(gidx0+(rid

UOp is an abstract syntax tree to represent computation

```python
class UOp:
  op: Ops
  dtype: dtypes
  src: tuple(UOp)
  arg: None
```

op field is operation

dtype is the data type

src is the parents of this node

arg is the argument of this node

In [8]:
from tinygrad.renderer.cstyle import MetalRenderer
from tinygrad.uop.ops import UOp, Ops
from tinygrad import dtypes

const = UOp(Ops.CONST, dtypes.float, arg=1.0)
add = UOp(Ops.ADD, dtypes.float, src=(const, const), arg=None)

print(add)

UOp(Ops.ADD, dtypes.float, arg=None, src=(
  x0:=UOp(Ops.CONST, dtypes.float, arg=1.0, src=()),
   x0,))


In [9]:
print(MetalRenderer().render([
    const,
    add
]))

#include <metal_stdlib>
using namespace metal;
kernel void test(uint3 gid [[threadgroup_position_in_grid]], uint3 lid [[thread_position_in_threadgroup]]) {
  float alu0 = (1.0f+1.0f);
}


In [10]:
print(MetalRenderer().render([
  UOp(Ops.SPECIAL, dtypes.int, arg=("gidx0", 16))
]))

#include <metal_stdlib>
using namespace metal;
kernel void test(uint3 gid [[threadgroup_position_in_grid]], uint3 lid [[thread_position_in_threadgroup]]) {
  int gidx0 = gid.x; /* 16 */
}


# Pattern matcher

It expresses the entire computation intoa nested tree and then recognizes parts which can be optimized.

In [13]:
from tinygrad import Tensor

with Context(DEBUG=4, NOOPT=True):
    a = Tensor.empty(4,4)
    b = a + 1

    b.realize()

#include <metal_stdlib>
using namespace metal;
kernel void E_16n1(device float* data0_16, device float* data1_16, uint3 gid [[threadgroup_position_in_grid]], uint3 lid [[thread_position_in_threadgroup]]) {
  int gidx0 = gid.x; /* 16 */
  float val0 = (*(data1_16+gidx0));
  *(data0_16+gidx0) = (val0+1.0f);
}
*** METAL      7 E_16n1                                       arg  2 mem  0.00 GB tm      7.25us/     0.04ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__add__', 'empty']


This output is unoptimized as one can see it is running not in parallel. The code is generated by looking at the AST which is the tree of UOps.

This is the code for how the code is generated from the UOps in the AST.

```python
patterns = [
    (STORE, lambda uop: "="),
    (CONST, lambda uop: f" {uop.arg} "),
    (ADD, lambda uop: f" + "),
]

def render_code(uop):
    code = []
    for _uop in uop:
        if uop.op == pattern[0]:
            _code = pattern[1](_uop)
            code.append(_code)
```

See above how the tuples store lambda functions to generate the code dynamically based on the UOp specifics like the arg.

The lingo:
The class PatternMatcher receives the list as init args.
Each tuple in list is of class UPat and it has op, dtype, src, args.

UOp tree converted to UPat tree.
UPat is a minimal abstraction of UOp. 

See here
```python
UOp(Ops.STORE, dtypes.void, arg=None, src=(
    UOp(Ops.DEFINE_GLOBAL, dtypes.float.ptr(), arg=0, src=()),
    ...
    UOp(Ops.ADD, dtypes.float, arg=None, src=(
      ...
    )
)

# is converted to:

UPat(Ops.STORE, name="x", src=(UPat.var("define_global"), UPat.var("addition")), lambda x, define_global, addition: ... )


Using the UPat tree, the PatternMatcher can now match the patterns and return the code generation once it sees the pattern it is looking for. This pattern can be complicated nested operations.

AST can be changed for optimization by changing lambda function: It can return another UOp instead of a string of code.

The optimized pattern matching tuple would look different than previous unoptimized. It would look like this: 

```python
  (UPat(Ops.VECTORIZE, name="x"),
   lambda ctx,x: f"{ctx.float4.replace('float4', ctx.render_dtype(x.dtype))}" + \
    (f"{{{','.join([ctx[y] for y in x.src])}}}" if ctx.device == "CLANG" else f"({','.join([ctx[y] for y in x.src])})")),
```


For our cases, you could generate new UOp from old one and then from that new one generate code. Would need to chagnge UOp to this for vectorized optimization:

```python
UOp(Ops.STORE, dtypes.void, arg=None, src=(
    UOp(Ops.INDEX, dtypes.float.ptr().vec(4), arg=None, src=(
      UOp(Ops.VECTORIZE, dtypes.float.ptr().vec(4), arg=None, src=(
        x3:=UOp(Ops.DEFINE_GLOBAL, dtypes.float.ptr(), arg=0, src=()),
         x3,
         x3,
         x3,)),
```

In [ ]:
from tinygrad import dtypes
from tinygrad.uop.ops import UOp, Ops
from tinygrad.renderer.cstyle import MetalRenderer

metal_renderer = MetalRenderer()
const = UOp(Ops.CONST, dtypes.float, arg=1.0)
define_global = UOp(Ops.DEFINE_GLOBAL, dtypes.float.ptr(), arg=0)
special = UOp(Ops.SPECIAL, dtypes.int, arg=('gidx0', 16), src=())
added = UOp(Ops.ADD, dtypes.long, arg=None, src=(define_global, special))
store = UOp(Ops.STORE, dtypes.void, arg=None, src=(added, const))
uops = [const, define_global, special, added, store]

rendered = metal_renderer.render(uops)
print(rendered)

#include <metal_stdlib>
using namespace metal;
kernel void test(device float* data0, uint3 gid [[threadgroup_position_in_grid]], uint3 lid [[thread_position_in_threadgroup]]) {
  int gidx0 = gid.x; /* 16 */
  *(data0+gidx0) = 1.0f;
}


In [20]:
from tinygrad.uop.ops import PatternMatcher, UPat

const_1 = UOp(Ops.CONST, dtypes.float, arg=0.5)
const_2 = UOp(Ops.CONST, dtypes.float, arg=0.5)

matcher = PatternMatcher([
  (UPat(Ops.CONST, dtypes.float, name="x"), lambda ctx, x: UOp(Ops.ADD, dtypes.float, src=(const_1, const_2))),
])

const = UOp(Ops.CONST, dtypes.float, arg=1.0)
const_rewritten = matcher.rewrite(const)


Above is a simple example of how a UOp could be rewritten as another UOp using a pattern matcher but to make work with rest of script bc of llinearization requirement, the consts must be added to the `uops` array.

In [21]:
from tinygrad import dtypes
from tinygrad.uop.ops import UOp, Ops
from tinygrad.renderer.cstyle import MetalRenderer

from tinygrad.uop.ops import PatternMatcher, UPat

const_1 = UOp(Ops.CONST, dtypes.float, arg=0.5)
const_2 = UOp(Ops.CONST, dtypes.float, arg=0.5)

matcher = PatternMatcher([
  (UPat(Ops.CONST, dtypes.float, name="x"), lambda ctx, x: UOp(Ops.ADD, dtypes.float, src=(const_1, const_2))),
])

metal_renderer = MetalRenderer()
const = UOp(Ops.CONST, dtypes.float, arg=1.0)

const_rewritten = matcher.rewrite(const)
define_global = UOp(Ops.DEFINE_GLOBAL, dtypes.float.ptr(), arg=0)
special = UOp(Ops.SPECIAL, dtypes.int, arg=('gidx0', 16), src=())
added = UOp(Ops.ADD, dtypes.long, arg=None, src=(define_global, special))
store = UOp(Ops.STORE, dtypes.void, arg=None, src=(added, const_rewritten))
uops = [const_1, const_2, const_rewritten, define_global, special, added, store]

rendered = metal_renderer.render(uops)
print(rendered)

#include <metal_stdlib>
using namespace metal;
kernel void test(device float* data0, uint3 gid [[threadgroup_position_in_grid]], uint3 lid [[thread_position_in_threadgroup]]) {
  int gidx0 = gid.x; /* 16 */
  *(data0+gidx0) = (0.5f+0.5f);
}


Why does the meta_renderer.render(uops) takes the uops list as an argument? Why the list? Why not just the new const UOp??? What does it do exactly? Does it need to know the definitions of the other UOps somehow?

# Shape Tracker

When matrix is stored linearly in memory need way to keep track of matrix shape!

An example is row_number * num_columns + column_number as index

num_columns here is referred to as stride.

Stride is the index number jumped when incrementing the dimensional index (e.g. row or column number)

In case of simple 2x2 matrix:
1 2
3 4
stored as: 1 2 3 4
the stride of the rows is 2 and stride for the columns is 1.

```python
View.create(shape=(2,2), strides=(2,1))
```

Will scale if think of it as 0th and 1st dimensions instead of as row and columns.

Can use this to transpose a matrix by flipping the strides. The actual memory will remain unchanged but how it is interpreted will be different. This reduces computation.

In [24]:
from tinygrad.shape.view import View

a = View.create(shape=(2,2), strides=(2,1))

idx, valid = a.to_indexed_uops()

print(idx)

print(idx.render())

UOp(Ops.ADD, dtypes.int, arg=None, src=(
  UOp(Ops.ADD, dtypes.int, arg=None, src=(
    UOp(Ops.CONST, dtypes.int, arg=0, src=()),
    UOp(Ops.MUL, dtypes.int, arg=None, src=(
      UOp(Ops.RANGE, dtypes.int, arg=0, src=(
        x4:=UOp(Ops.CONST, dtypes.int, arg=2, src=()),)),
       x4,)),)),
  UOp(Ops.MUL, dtypes.int, arg=None, src=(
    UOp(Ops.RANGE, dtypes.int, arg=1, src=(
       x4,)),
    UOp(Ops.CONST, dtypes.int, arg=1, src=()),)),))
((ridx0*2)+ridx1)


This is the same as earlier eqn. For computing memory index. See the bottom line of output for the more concise eqn not written in UOps.

Can broadcast a vector easily by using a stride of 0 to fill in shape with the same values as in the 0th index of a dimension.

Also, the shape tracker can be used for matrixes which are stored in memory which is not continuous. But idk how??

In [25]:
a = View.create(shape=(3,2), strides=(2,1))

a = a.permute((1,0))
print(a.shape)
print(a.strides)

a = a.reshape((3,2))
print(a)

(2, 3)
(1, 2)
None


The returned value is none because a single view can't be reshaped because shape from memory to matrix can no longer be written with just shape and stride: 

```
 (3,2)                        (2,3)                    (3,2)
0x00 0x01   transpose    0x00 0x02 0x04  reshape     0x00 0x02
0x02 0x03 ------------>  0x01 0x03 0x05 ---------->  0x04 0x01
0x04 0x05                                            0x03 0x05
```

Shapetracker is an object which can hold multiple views to track shape even in cases such as those.

In [26]:
from tinygrad.shape.shapetracker import ShapeTracker

a = ShapeTracker.from_shape((3,2))
a = a.permute((1,0))
a = a.reshape((3,2))
print(a)

ShapeTracker(views=(View(shape=(2, 3), strides=(1, 2), offset=0, mask=None, contiguous=False), View(shape=(3, 2), strides=(2, 1), offset=0, mask=None, contiguous=True)))


Here you can see the views are inputted. 

In [27]:
idx, valid = a.to_indexed_uops()
print(idx.render())

(((((ridx0*2)+ridx1)%3)*2)+(((ridx0*2)+ridx1)//3))


It gets this eqn by rendering the last view and then project each element onto the first one (does this continually with several views).

In [ ]:
def views_to_indexed_uops(views: Tuple[View, ...], _idxs:Optional[Tuple[UOp, ...]]=None) -> Tuple[UOp, UOp]:
  idx, valid = views[-1].to_indexed_uops(_idxs)
  for view in reversed(views[0:-1]):
    view = view.minify()
    acc, idxs = 1, []
    for d in reversed(view.shape):
      idxs.append((idx//acc)%d)
      acc *= d
    idx, valid = view.to_indexed_uops(idxs[::-1], valid)
  return idx, valid

First the expression for the last view is rendered: `row*2+col`, then the relation between this view and previous view is `(expression // acc) % shape` where shape is every shape in the previous view and and the acc is accumulated

<img src="img/91.png">

See the diagram above explaining and breaking down why the formula works. Basically the flattened versions of the matrix before and after reshaping are the same, so if you let their index determining expressions equal to each other you can solve for the indexes in terms of each other and using the logical relation of indexes for transpose and the final flatten, one can determine the memory indexes in terms of the first.

I still need to spend more time understanding the actual code though!

# Matmul

Matrix multiplications are either reduction operations where number of dimensions decrease or elementwise operations where the number of dimensions stays the same. 

Note: Matrixes only get bigger via broadcasting.

In tinygrad reduce operations create copys in new memory and elementwise operations change existing memory or shapetracker.

All operations can be represented as combinations of these two operations.

Examples: 

```python
def mean(self, axis):
  summed = self.reduce(ADD, axis)
  divided = summed.elementwise(DIVIDE, self.shape[axis])
  return divided
```

```python
def max(self, axis):
  return self.reduce(MAX, axis)
```

```python
def softmax(self, axis):
  _max = self.max(axis)
  subtracted = self.elementwise(MINUS, _max)
  exp = self.elementwise(EXP)
  return exp
```

How code is generated sort of:

```python
def elementwise(self, op, other):
  return f"""
float res = self {op} other;
  """
```

```python
def reduce(self, op, axis):
  return f"""
float acc = 0;
for (int i = 0; i < 16; i++) {
  acc = {op}(acc, val);
}
return acc;
"""
```

Here is it fr:
```python
def elementwise(self, op, other):
  shape = self.shape
  strides = self.strides
  index = f"x * {strides[0]} + y * {strides[1]}"
  return f"""
void elementwise(float* data0, float* data1, float*, data2) {
  int x = threadIdx.x;
  int y = threadIdx.y;
  data0[{index}] = data1[{index}] {op} data2[{index}]; 
}
"""
```


Can do a bunch of ShapeTracker changes utilizing broadcasting and transpose then an elementwise multiplication and a sum reduce operation is how you could perform a standard dot product utilizing just